# Climate Data Scraping: Download Data
#### Fire Atlas Dataset + NCDC GSOM + NCDC Monthly Normals

Andrew McDonald // CSE 847 // April 2021

In [1]:
import requests
import urllib
import json
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 25)

In [2]:
ignitions = pd.read_csv("data/usa_2016_wildfires_stations.csv")
ignitions

,fire_ID,latitude,longitude,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,...,normal_elevation,normal_mindate,normal_maxdate,normal_latitude,normal_datacoverage,normal_id,normal_elevationUnit,normal_longitude,distance_from_gsom,distance_from_normal
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,...,15.5,2010-01-01,2010-12-01,20.89972,1,GHCND:USW00022516,METERS,-156.42861,128.450424,6.400746
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,...,15.5,2010-01-01,2010-12-01,20.89972,1,GHCND:USW00022516,METERS,-156.42861,128.162642,6.747153
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,...,15.5,2010-01-01,2010-12-01,20.89972,1,GHCND:USW00022516,METERS,-156.42861,125.720919,9.916521
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,...,923.5,2010-01-01,2010-12-01,20.70090,1,GHCND:USC00515004,METERS,-156.35620,100.848205,14.008401
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,...,6.1,2010-01-01,2010-12-01,19.72810,1,GHCND:USC00513911,METERS,-156.05880,17.928047,19.225533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11519,56789,40.8521,-78.1691,0.43,2.78,2016-04-14,105,2016-04-14,105,1,...,524.3,2010-01-01,2010-12-01,40.87291,1,GHCND:USC00366921,METERS,-78.21615,4.590851,4.590851
11520,56790,40.4188,-76.7056,0.64,3.70,2015-11-14,318,2015-11-14,318,1,...,95.1,2010-01-01,2010-12-01,40.19620,1,GHCND:USW00014711,METERS,-76.77240,5.937829,25.361543
11521,56791,40.0271,-74.4895,0.86,4.63,2016-06-26,178,2016-07-04,186,9,...,18.3,2010-01-01,2010-12-01,39.97083,1,GHCND:USC00286843,METERS,-74.68278,27.321358,17.648146
11522,56792,40.0063,-74.4994,3.00,9.26,2016-06-13,165,2016-06-19,171,7,...,18.3,2010-01-01,2010-12-01,39.97083,1,GHCND:USC00286843,METERS,-74.68278,29.402474,16.149717


In [3]:
u_gsom_stations = ignitions["gsom_id"].unique()
u_normal_stations = ignitions["normal_id"].unique()
u_gsom_stations.shape, u_normal_stations.shape

((1511,), (1483,))

## 1. Obtain GSOM Data

In [4]:
api_token = "syKpWfFaNGhSsksXrJhcrwQspIYihUqW"
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
headers = {"token": api_token}
resource = "data?"

In [5]:
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-12-31",
    "units": "metric",
    "limit": "1000"
}

gsom = []
for i, station in enumerate(u_gsom_stations):
    query["stationid"] = station
    query_str = urllib.parse.urlencode(query, doseq=False)
    endpoint = base_url + resource + query_str
    response = requests.get(endpoint, headers=headers)
    results = json.loads(response.text)
    if "results" in results:
        gsom.append(results["results"])
    else:
        print(f"No data for station {station}.")

    if i % 100 == 0:
        print(f"Finished station {i}")

Finished station 0
No data for station GHCND:USC00266746.
Finished station 100
No data for station GHCND:USC00021306.
No data for station GHCND:USC00023393.
No data for station GHCND:USC00107706.
Finished station 200
No data for station GHCND:USC00103558.
Finished station 300
No data for station GHCND:USC00055116.
No data for station GHCND:USC00422828.
No data for station GHCND:USC00057337.
Finished station 400
Finished station 500
No data for station GHCND:USC00414792.
Finished station 600
Finished station 700
No data for station GHCND:USC00396170.
No data for station GHCND:USC00395285.
Finished station 800
No data for station GHCND:USC00131833.
No data for station GHCND:USC00144665.
Finished station 900
Finished station 1000
No data for station GHCND:USC00227921.
Finished station 1100
Finished station 1200
No data for station GHCND:USC00210515.
No data for station GHCND:USC00216228.
Finished station 1300
No data for station GHCND:USC00131731.
Finished station 1400
Finished station 15

In [6]:
rows = []
for station in gsom:
    # datetime.fromisoformat(entry["date"]).month
    i = 0
    row = {"date": None}
    for i, variable in enumerate(station):
        if row["date"] != variable["date"]:
            if i > 0:
                rows.append(row)
            row = {"date": variable["date"], "station": variable["station"]}
        row[variable["datatype"]] = variable["value"]

In [8]:
# clean up scraped data
gsom_df = pd.DataFrame(rows)
gsom_df["date"] = gsom_df["date"].apply(datetime.fromisoformat)
for i, row in gsom_df.iterrows():
    for col in ["DYNT", "DYSN", "DYXP", "DYXT", "DYSD"]:
        if not pd.isna(row[col]):
            s = str(row[col])[:-2]
            y, m = int(s[:4]), int(s[4:6])
            d = int(s[6:]) if len(s) > 6 else 1
            gsom_df.loc[i, col] = datetime(y, m, d).date()

with pd.option_context("display.max_columns", None):
    display(gsom_df)

,date,station,AWND,CDSD,CLDD,DP01,DP10,DP1X,DSNW,DT00,DT32,DX32,DX70,DX90,DYNT,DYSN,DYXP,DYXT,EMNT,EMSN,EMXP,EMXT,HDSD,HTDD,PRCP,SNOW,TAVG,TMAX,TMIN,WDF2,WSF2,DYFG,WDF5,WSF5,DSND,DYSD,EMSD,DYHF,DYTS,EVAP,MNPN,MXPN,WDMV,HN01,HX01,LN01,LX01,MN01,MX01,WSFG,HN02,HX02,LN02,LX02,MN02,MX02,WDFG,HN03,HX03,LN03,LX03,MN03,MX03,HN04,HX04,LN04,LX04,MN04,MX04
0,2016-01-01,GHCND:USW00021510,3.1,188.6,188.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,2016-01-15,2016-01-31,2016-01-26,2016-01-24,19.4,0.0,0.0,28.9,0.0,0.0,0.0,0.0,24.4,27.6,21.2,280.0,41.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-02-01,GHCND:USW00021510,3.3,360.2,171.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,2016-02-08,2016-02-29,2016-02-07,2016-02-16,16.7,0.0,0.0,30.6,0.0,0.0,0.0,0.0,24.3,28.1,20.4,360.0,16.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-03-01,GHCND:USW00021510,3.8,567.1,206.9,NaN,NaN,NaN,NaN,0.0,0.0,0.0,31.0,0.0,2016-03-24,NaN,NaN,2016-03-09,16.7,NaN,NaN,31.1,0.0,0.0,NaN,NaN,25.0,28.8,21.2,360.0,16.1,4.0,350.0,19.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-04-01,GHCND:USW00021510,3.4,797.9,230.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,2016-04-01,2016-04-30,2016-04-29,2016-04-15,21.1,0.0,0.0,30.6,0.0,0.0,0.0,0.0,26.0,29.3,22.7,350.0,13.4,3.0,340.0,16.1,0.0,2016-04-30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-05-01,GHCND:USW00021510,3.7,1041.8,243.9,6.0,3.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,2016-05-12,2016-05-31,2016-05-24,2016-05-20,21.1,0.0,6.6,31.1,0.0,0.0,14.8,0.0,26.2,29.4,23.0,340.0,10.7,7.0,350.0,13.4,0.0,2016-05-31,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15719,2016-07-01,GHCND:USC00283951,NaN,353.3,206.8,12.0,11.0,2.0,NaN,0.0,0.0,0.0,31.0,17.0,2016-07-13,NaN,2016-07-26,2016-07-26,13.3,NaN,50.5,36.1,0.0,0.0,206.8,NaN,25.0,31.8,18.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15720,2016-08-01,GHCND:USC00283951,NaN,540.8,187.5,8.0,5.0,0.0,0.0,0.0,0.0,0.0,31.0,16.0,2016-08-23,2016-08-31,2016-08-01,2016-08-14,9.4,0.0,11.9,37.2,0.3,0.3,32.2,0.0,24.4,31.6,17.2,NaN,NaN,NaN,NaN,NaN,0.0,2016-08-31,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15721,2016-09-01,GHCND:USC00283951,NaN,619.4,78.6,7.0,5.0,0.0,NaN,0.0,0.0,0.0,29.0,5.0,2016-09-26,NaN,2016-09-20,2016-09-11,4.4,NaN,19.1,36.1,27.9,27.6,48.2,NaN,20.1,27.9,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15722,2016-10-01,GHCND:USC00283951,NaN,631.3,11.9,9.0,7.0,1.0,0.0,0.0,6.0,0.0,12.0,0.0,2016-10-30,2016-10-31,2016-10-31,2016-10-20,-1.7,0.0,27.9,30.0,206.7,178.8,92.5,0.0,12.9,19.9,6.0,NaN,NaN,NaN,NaN,NaN,0.0,2016-10-31,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# preview counts and sparsity
with pd.option_context("display.max_columns", None):
    display(gsom_df.describe())

,AWND,CDSD,CLDD,DP01,DP10,DP1X,DSNW,DT00,DT32,DX32,DX70,DX90,EMNT,EMSN,EMXP,EMXT,HDSD,HTDD,PRCP,SNOW,TAVG,TMAX,TMIN,WDF2,WSF2,DYFG,WDF5,WSF5,DSND,EMSD,DYHF,DYTS,EVAP,MNPN,MXPN,WDMV,HN01,HX01,LN01,LX01,MN01,MX01,WSFG,HN02,HX02,LN02,LX02,MN02,MX02,WDFG,HN03,HX03,LN03,LX03,MN03,MX03,HN04,HX04,LN04,LX04,MN04,MX04
count,2176.000000,13010.000000,15009.000000,15095.000000,15095.000000,15095.000000,12280.000000,15048.000000,15048.000000,15109.000000,15109.000000,15109.000000,15048.000000,12280.000000,15095.000000,15109.000000,12786.000000,15009.000000,15376.000000,12280.000000,15019.000000,15109.000000,15048.000000,2087.000000,2087.000000,4162.000000,2061.000000,2061.000000,12146.000000,12146.000000,1235.000000,3737.000000,479.000000,168.000000,166.000000,339.000000,528.000000,554.000000,528.000000,554.000000,528.000000,554.000000,69.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,53.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.00000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,3.337270,296.190699,67.887048,7.247168,4.551043,0.671481,0.481759,0.417398,7.765284,1.311867,17.083923,4.767357,0.005722,19.358306,26.036549,28.456033,1215.010449,191.734819,68.514282,38.486319,14.190678,21.067106,7.339753,220.215621,14.375084,7.113647,219.150898,19.107181,2.337724,40.476535,3.065587,4.452770,192.847599,13.483929,26.452410,3337.460177,19.570644,24.203069,11.743561,15.193863,15.610417,19.853971,27.156522,16.328947,21.652632,7.818421,11.097368,12.103947,16.553947,197.547170,15.120833,18.866667,6.583333,9.287500,11.220833,13.91250,15.209091,17.163636,7.254545,9.754545,11.690909,13.227273
std,1.203796,449.233210,99.413097,4.636222,3.483749,1.076135,1.406126,2.058869,10.582318,4.263033,12.098450,8.362854,11.312550,52.895608,27.518643,8.054156,1411.607343,215.399595,71.918703,129.368253,9.453317,9.726460,9.486156,96.165214,3.568401,6.730705,94.846360,4.590854,6.912025,158.961028,2.563882,4.372324,159.642972,5.673875,6.048518,2457.274135,8.268547,10.097881,8.910589,9.612573,8.640836,9.996952,21.113652,9.048754,10.739664,8.598492,9.554977,8.772828,10.358461,110.762144,8.865810,10.020660,7.704920,8.298471,8.266274,9.27232,8.009926,8.143129,7.906878,8.568823,8.051516,8.397987
min,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-42.100000,0.000000,0.000000,-13.900000,0.000000,0.000000,0.000000,0.000000,-21.900000,-19.400000,-25.400000,10.000000,4.000000,1.000000,10.000000,6.300000,0.000000,0.000000,1.000000,1.000000,0.000000,2.600000,8.800000,0.000000,-3.900000,-3.900000,-10.600000,-10.600000,-5.000000,-7.000000,7.600000,-1.100000,0.000000,-11.700000,-5.000000,-3.900000,-1.900000,8.000000,0.000000,1.100000,-5.000000,-3.300000,-1.100000,-0.30000,1.700000,3.900000,-3.900000,-1.700000,-0.300000,0.400000
25%,2.500000,0.000000,0.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,-6.700000,0.000000,7.900000,23.900000,25.700000,8.300000,16.800000,0.000000,7.600000,14.400000,0.700000,160.000000,11.600000,2.000000,160.000000,16.100000,0.000000,0.000000,1.000000,1.000000,127.000000,9.100000,22.225000,1683.500000,13.900000,17.200000,3.900000,7.800000,8.775000,12.750000,16.100000,9.850000,14.275000,0.975000,2.800000,4.875000,9.200000,70.000000,8.900000,11.000000,0.300000,1.550000,4.650000,7.07500,11.100000,12.500000,1.100000,2.800000,6.850000,8.200000
50%,3.200000,50.300000,10.400000,7.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,19.000000,0.000000,0.000000,0.000000,18.300000,30.000000,619.150000,114.200000,46.800000,0.000000,14.900000,22.300000,7.500000,240.000000,13.900000,4.000000,230.000000,18.800000,0.000000,0.000000,2.000000,3.000000,174.000000,13.550000,27.150000,2827.000000,20.600000,25.000000,11.700000,15.600000,16.000000,20.600000,20.100000,18.300000,23.300000,6.700000,10.850000,11.500000,17.250000,235.000000,16.700000,21.700000,6.700000,10.300000,12.450000,16.20000,16.700000,20.000000,8.90000

In [16]:
# save data
gsom_df.to_csv("data/ncdc_gsom_data.csv", index=False)

## 2. Obtain Monthly Normal Data


In [11]:
query = {
    "datasetid": "NORMAL_MLY",
    "startdate": "2010-01-01",
    "enddate": "2010-12-31",
    "units": "metric",
    "limit": "1000"
}

normal = []
for i, station in enumerate(u_normal_stations):
    query["stationid"] = station
    query_str = urllib.parse.urlencode(query, doseq=False)
    endpoint = base_url + resource + query_str
    response = requests.get(endpoint, headers=headers)
    results = json.loads(response.text)
    if "results" in results:
        normal.append(results["results"])
    else:
        print(f"No data for station {station}.")

    if i % 100 == 0:
        print(f"Finished station {i}")

Finished station 0
Finished station 100
Finished station 200
Finished station 300
Finished station 400
Finished station 500
Finished station 600
Finished station 700
Finished station 800
Finished station 900
Finished station 1000
Finished station 1100
Finished station 1200
Finished station 1300
Finished station 1400


In [12]:
rows = []
for station in normal:
    # datetime.fromisoformat(entry["date"]).month
    i = 0
    row = {"date": None}
    for i, variable in enumerate(station):
        if row["date"] != variable["date"]:
            if i > 0:
                rows.append(row)
            row = {"date": variable["date"], "station": variable["station"]}
        row[variable["datatype"]] = variable["value"]

In [13]:
# clean up scraped data
normal_df = pd.DataFrame(rows)
normal_df["date"] = normal_df["date"].apply(datetime.fromisoformat)
with pd.option_context("display.max_columns", None):
    display(normal_df)

,date,station,MLY-CLDD-BASE45,MLY-CLDD-BASE50,MLY-CLDD-BASE55,MLY-CLDD-BASE57,MLY-CLDD-BASE60,MLY-CLDD-BASE70,MLY-CLDD-BASE72,MLY-CLDD-NORMAL,MLY-DUTR-NORMAL,MLY-DUTR-STDDEV,MLY-GRDD-BASE40,MLY-GRDD-BASE45,MLY-GRDD-BASE50,MLY-GRDD-BASE55,MLY-GRDD-BASE57,MLY-GRDD-BASE60,MLY-GRDD-BASE65,MLY-GRDD-BASE70,MLY-GRDD-BASE72,MLY-GRDD-TB4886,MLY-GRDD-TB5086,MLY-HTDD-BASE40,MLY-HTDD-BASE45,MLY-HTDD-BASE50,MLY-HTDD-BASE55,MLY-HTDD-BASE57,MLY-HTDD-BASE60,MLY-HTDD-NORMAL,MLY-PRCP-25PCTL,MLY-PRCP-50PCTL,MLY-PRCP-75PCTL,MLY-PRCP-AVGNDS-GE001HI,MLY-PRCP-AVGNDS-GE010HI,MLY-PRCP-AVGNDS-GE050HI,MLY-PRCP-AVGNDS-GE100HI,MLY-PRCP-NORMAL,MLY-SNOW-25PCTL,MLY-SNOW-50PCTL,MLY-SNOW-75PCTL,MLY-SNOW-AVGNDS-GE001TI,MLY-SNOW-AVGNDS-GE010TI,MLY-SNOW-AVGNDS-GE030TI,MLY-SNOW-AVGNDS-GE050TI,MLY-SNOW-AVGNDS-GE100TI,MLY-SNOW-NORMAL,MLY-SNWD-AVGNDS-GE001WI,MLY-SNWD-AVGNDS-GE003WI,MLY-SNWD-AVGNDS-GE005WI,MLY-SNWD-AVGNDS-GE010WI,MLY-TAVG-NORMAL,MLY-TAVG-STDDEV,MLY-TMAX-AVGNDS-GRTH040,MLY-TMAX-AVGNDS-GRTH050,MLY-TMAX-AVGNDS-GRTH060,MLY-TMAX-AVGNDS-GRTH070,MLY-TMAX-AVGNDS-GRTH080,MLY-TMAX-AVGNDS-GRTH090,MLY-TMAX-AVGNDS-GRTH100,MLY-TMAX-AVGNDS-LSTH032,MLY-TMAX-NORMAL,MLY-TMAX-STDDEV,MLY-TMIN-AVGNDS-LSTH000,MLY-TMIN-AVGNDS-LSTH010,MLY-TMIN-AVGNDS-LSTH020,MLY-TMIN-AVGNDS-LSTH032,MLY-TMIN-AVGNDS-LSTH040,MLY-TMIN-AVGNDS-LSTH050,MLY-TMIN-AVGNDS-LSTH060,MLY-TMIN-AVGNDS-LSTH070,MLY-TMIN-NORMAL,MLY-TMIN-PRBOCC-LSTH016,MLY-TMIN-PRBOCC-LSTH020,MLY-TMIN-PRBOCC-LSTH024,MLY-TMIN-PRBOCC-LSTH028,MLY-TMIN-PRBOCC-LSTH032,MLY-TMIN-PRBOCC-LSTH036,MLY-TMIN-STDDEV
0,2010-01-01,GHCND:USW00022516,465.0,379.0,293.0,258.0,207.0,40.0,18.0,103.0,-8.2,-16.7,551.0,465.0,379.0,293.0,258.0,207.0,121.0,40.0,18.0,395.0,361.0,0.0,0.0,0.0,0.0,0.0,0.0,-4338.0,25.65,58.42,95.50,9.6,4.5,1.5,0.7,72.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.2,-17.1,31.0,31.0,31.0,30.9,19.0,0.0,0.0,0.0,27.0,-16.8,0.0,0.0,0.0,0.0,0.0,0.0,7.2,29.3,17.4,0,0,0,0,0,0,-16.9
1,2010-02-01,GHCND:USW00022516,418.0,340.0,262.0,231.0,184.0,35.0,16.0,89.0,-7.9,-16.7,496.0,418.0,340.0,262.0,231.0,184.0,107.0,35.0,16.0,353.0,322.0,0.0,0.0,0.0,0.0,0.0,0.0,-4338.0,17.53,27.18,51.56,8.5,3.6,0.9,0.4,48.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.1,-17.1,28.0,28.0,28.0,27.9,17.7,0.0,0.0,0.0,27.1,-16.9,0.0,0.0,0.0,0.0,0.0,-777.7,7.5,26.9,17.2,0,0,0,0,0,0,-16.8
2,2010-03-01,GHCND:USW00022516,481.0,394.0,308.0,274.0,222.0,53.0,26.0,118.0,-8.3,-16.7,567.0,481.0,394.0,308.0,274.0,222.0,136.0,53.0,26.0,411.0,376.0,0.0,0.0,0.0,0.0,0.0,0.0,-4338.0,22.61,47.75,100.08,10.3,4.4,1.6,0.6,62.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.7,-17.1,31.0,31.0,31.0,31.0,21.3,-777.7,0.0,0.0,27.4,-16.8,0.0,0.0,0.0,0.0,0.0,0.0,4.5,29.6,18.0,0,0,0,0,0,0,-17.1
3,2010-04-01,GHCND:USW00022516,484.0,401.0,318.0,284.0,234.0,68.0,38.0,133.0,-8.5,-16.8,568.0,484.0,401.0,318.0,284.0,234.0,151.0,68.0,38.0,415.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0,-18.0,7.87,22.86,34.80,9.2,2.9,0.8,0.3,39.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.3,-17.1,30.0,30.0,30.0,30.0,23.8,-777.7,0.0,0.0,28.0,-16.8,0.0,0.0,0.0,0.0,0.0,0.0,1.8,27.4,18.7,0,0,0,0,0,0,-16.9
4,2010-05-01,GHCND:USW00022516,529.0,443.0,357.0,323.0,271.0,99.0,66.0,167.0,-8.2,-16.8,616.0,529.0,443.0,357.0,323.0,271.0,185.0,99.0,66.0,456.0,422.0,0.0,0.0,0.0,0.0,0.0,0.0,-4338.0,2.03,12.45,24.13,6.3,1.8,0.3,0.1,18.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.3,-17.1,31.0,31.0,31.0,31.0,28.6,0.4,0.0,0.0,29.1,-16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.4,24.5,19.5,0,0,0,0,0,0,-17.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,2010-07-01,GHCND:USC00286843,525.0,439.0,353.0,318.0,267.0,101.0,74.0,163.0,-4.3,-16.6,611.0,525.0,439.0,353.0,318.0,267.0,181.0,101.0,74.0,429.0,394.0,0.0,0.0,0.0,0.0,-4321.0,-4321.0,-17

In [14]:
# preview counts and sparsity
with pd.option_context("display.max_columns", None):
    display(normal_df.describe())

,MLY-CLDD-BASE45,MLY-CLDD-BASE50,MLY-CLDD-BASE55,MLY-CLDD-BASE57,MLY-CLDD-BASE60,MLY-CLDD-BASE70,MLY-CLDD-BASE72,MLY-CLDD-NORMAL,MLY-DUTR-NORMAL,MLY-DUTR-STDDEV,MLY-GRDD-BASE40,MLY-GRDD-BASE45,MLY-GRDD-BASE50,MLY-GRDD-BASE55,MLY-GRDD-BASE57,MLY-GRDD-BASE60,MLY-GRDD-BASE65,MLY-GRDD-BASE70,MLY-GRDD-BASE72,MLY-GRDD-TB4886,MLY-GRDD-TB5086,MLY-HTDD-BASE40,MLY-HTDD-BASE45,MLY-HTDD-BASE50,MLY-HTDD-BASE55,MLY-HTDD-BASE57,MLY-HTDD-BASE60,MLY-HTDD-NORMAL,MLY-PRCP-25PCTL,MLY-PRCP-50PCTL,MLY-PRCP-75PCTL,MLY-PRCP-AVGNDS-GE001HI,MLY-PRCP-AVGNDS-GE010HI,MLY-PRCP-AVGNDS-GE050HI,MLY-PRCP-AVGNDS-GE100HI,MLY-PRCP-NORMAL,MLY-SNOW-25PCTL,MLY-SNOW-50PCTL,MLY-SNOW-75PCTL,MLY-SNOW-AVGNDS-GE001TI,MLY-SNOW-AVGNDS-GE010TI,MLY-SNOW-AVGNDS-GE030TI,MLY-SNOW-AVGNDS-GE050TI,MLY-SNOW-AVGNDS-GE100TI,MLY-SNOW-NORMAL,MLY-SNWD-AVGNDS-GE001WI,MLY-SNWD-AVGNDS-GE003WI,MLY-SNWD-AVGNDS-GE005WI,MLY-SNWD-AVGNDS-GE010WI,MLY-TAVG-NORMAL,MLY-TAVG-STDDEV,MLY-TMAX-AVGNDS-GRTH040,MLY-TMAX-AVGNDS-GRTH050,MLY-TMAX-AVGNDS-GRTH060,MLY-TMAX-AVGNDS-GRTH070,MLY-TMAX-AVGNDS-GRTH080,MLY-TMAX-AVGNDS-GRTH090,MLY-TMAX-AVGNDS-GRTH100,MLY-TMAX-AVGNDS-LSTH032,MLY-TMAX-NORMAL,MLY-TMAX-STDDEV,MLY-TMIN-AVGNDS-LSTH000,MLY-TMIN-AVGNDS-LSTH010,MLY-TMIN-AVGNDS-LSTH020,MLY-TMIN-AVGNDS-LSTH032,MLY-TMIN-AVGNDS-LSTH040,MLY-TMIN-AVGNDS-LSTH050,MLY-TMIN-AVGNDS-LSTH060,MLY-TMIN-AVGNDS-LSTH070,MLY-TMIN-NORMAL,MLY-TMIN-PRBOCC-LSTH016,MLY-TMIN-PRBOCC-LSTH020,MLY-TMIN-PRBOCC-LSTH024,MLY-TMIN-PRBOCC-LSTH028,MLY-TMIN-PRBOCC-LSTH032,MLY-TMIN-PRBOCC-LSTH036,MLY-TMIN-STDDEV
count,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000,16313.000000
mean,170.750690,45.199963,-90.433581,-147.829522,-223.403053,-530.731502,-579.233372,-367.525593,-4.128180,-16.240385,275.578925,170.750690,45.199963,-90.433581,-147.829522,-223.403053,-349.770367,-530.731502,-579.233372,181.250782,149.946055,-517.560902,-494.248513,-446.202906,-339.359713,-282.501808,-201.246552,-90.715748,38.706511,63.336620,95.966418,7.424024,3.905640,-9.662343,-52.937449,70.034449,13.040501,28.358965,54.347018,-32.824974,-31.958499,-34.332624,-42.069963,-45.183994,-609.634463,-22.631637,-22.849237,-19.536278,-15.245154,14.148262,-16.135487,27.464078,23.534236,17.025654,9.440183,-3.276203,-20.089603,-27.382517,-22.809673,20.972684,-15.801520,-14.486158,-17.518292,-19.967707,-11.847459,-3.718298,5.167688,18.485374,27.508527,7.323374,219.735303,278.000123,346.651382,423.523509,503.266413,579.616257,-16.143940
std,711.209058,881.525949,1047.856118,1113.546190,1188.377552,1481.367276,1518.814110,1306.823908,2.955789,0.420082,584.197425,711.209058,881.525949,1047.856118,1113.546190,1188.377552,1307.067568,1481.367276,1518.814110,482.281736,521.563134,1462.221366,1464.590445,1448.021720,1366.517005,1314.049629,1237.504208,1113.285049,32.301529,45.085305,61.657529,9.242968,26.087032,93.863057,197.353394,79.840305,55.117814,92.364603,147.765619,158.323627,155.661391,160.206161,176.088289,181.956824,3530.266825,136.860017,135.832192,125.417105,110.285708,9.522364,0.636764,15.357271,38.128654,62.368163,78.983221,107.327623,136.986862,144.725370,1

In [17]:
# save data
normal_df.to_csv("data/ncdc_normal_mly_data.csv", index=False)
